In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


In [10]:
import os
os.chdir('../data')  # Change to your data directory

In [11]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [4]:
selected_features = [
    'Administrative',
    'Informational',
    'ProductRelated',
    'PageValues',
    'BounceRates',
    'ExitRates',
    'Month',
    'VisitorType',
    'Weekend',
    'TrafficType'
]


In [5]:
X = train_df[selected_features].copy()
y = train_df['Revenue']
X_test = test_df[selected_features].copy()

In [6]:
categorical_cols = ['Month', 'VisitorType', 'Weekend', 'TrafficType']
for col in categorical_cols:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

numeric_cols = [col for col in selected_features if col not in categorical_cols]

建立columnTransformers

In [7]:
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_cols),
    
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categorical_cols)
])

建模&訓練

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42)
}

results = {}
best_auc = -1
best_model = None
best_name = ""

#分割&訓練驗證
X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

#訓練&評估模型
results = {}
best_model = None
best_auc = -1
best_name = ""

for name, model in models.items():
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
    
    pipe.fit(X_train_raw, y_train)
    val_probs = pipe.predict_proba(X_val_raw)[:, 1]
    
    auc = roc_auc_score(y_val, val_probs)
    acc = accuracy_score(y_val, val_probs > 0.5)
    results[name] = {"AUC": auc, "Accuracy": acc}
    
    print(f"{name}: AUC = {auc:.4f}, Accuracy = {acc:.4f}")
    
    if auc > best_auc:
        best_auc = auc
        best_model = pipe
        best_name = name
 
print(f"\n Best Model: {best_name} (AUC = {best_auc:.4f})")

Logistic Regression: AUC = 0.8686, Accuracy = 0.8778
Random Forest: AUC = 0.9160, Accuracy = 0.8975


d:\airbnb\.conda\Lib\site-packages\xgboost\training.py:183: UserWarning: [11:13:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost: AUC = 0.9111, Accuracy = 0.8864
[LightGBM] [Info] Number of positive: 1022, number of negative: 5458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001249 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1041
[LightGBM] [Info] Number of data points in the train set: 6480, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.157716 -> initscore=-1.675321
[LightGBM] [Info] Start training from score -1.675321
LightGBM: AUC = 0.9195, Accuracy = 0.8907

 Best Model: LightGBM (AUC = 0.9195)


d:\airbnb\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# 訓練完的模型變數
final_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', models[best_name])
])
final_model.fit(X, y)  # 使用完整訓練資料進行訓練

# 預測完的儲存變數
test_probs = final_model.predict_proba(X_test)[:, 1]  # 預測消費的機率

# 將預測結果寫入 CSV，並按照 ID 排序
submission_df = test_df[['ID']].copy()  
submission_df['HasRevenue'] = test_probs 
submission_df = submission_df.sort_values('ID', ascending=True)  # 按 ID 排序
submission_df.to_csv('my_submission1.csv', index=False)  # 輸出 CSV 檔案


[LightGBM] [Info] Number of positive: 1277, number of negative: 6823
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003636 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1111
[LightGBM] [Info] Number of data points in the train set: 8100, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.157654 -> initscore=-1.675786
[LightGBM] [Info] Start training from score -1.675786
預測完成，輸出檔案：my_submission.csv


d:\airbnb\.conda\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
